<a href="https://colab.research.google.com/github/WasikulIslam1/CIS-4400-Project-9/blob/main/Location_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install azure-storage-blob
!pip install pyarrow

In [3]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient



Use of the free database in production requires that you link back to:
https://simplemaps.com/data/us-counties.

In [ ]:

# Specify the file path of your CSV file
file_path = "uscounties.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df.head()
df.drop(columns=['population','county','county_ascii'], inplace=True)


In [10]:
with open('config.json', 'r') as f:
    config = json.load(f)

connection_string = config['connectionString']


# Path to your local CSV file
container_name = 'location'
file_path = "uscounties.csv"
blob_name = "location_raw"

# Create a blob service client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Create a blob client using the container and blob name
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the file
with open(file_path, "rb") as data:
    blob_client.upload_blob(data)

In [12]:

new_column_names = {
    'county_full': 'county_name',
    'lat': 'latitude',
    'lng': 'longitude',
    # Add more mappings as needed
}

# Use the rename function with the dictionary
df.rename(columns=new_column_names, inplace=True)

csv_file_path = "Location_dim.csv"

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)
df.head(25)

,county_name,county_fips,state_id,state_name,latitude,longitude
0,Los Angeles County,6037,CA,California,34.3219,-118.2247
1,Cook County,17031,IL,Illinois,41.8401,-87.8168
2,Harris County,48201,TX,Texas,29.8578,-95.3938
3,Maricopa County,4013,AZ,Arizona,33.3490,-112.4915
4,San Diego County,6073,CA,California,33.0343,-116.7351
5,Orange County,6059,CA,California,33.7031,-117.7609
6,Miami-Dade County,12086,FL,Florida,25.6150,-80.5624
7,Kings County,36047,NY,New York,40.6413,-73.9383
8,Dallas County,48113,TX,Texas,32.7666,-96.7778
9,Riverside County,6065,CA,California,33.7437,-115.9938


In [ ]:
# Load Azure Storage connection string and container name from config.json
with open('config.json', 'r') as f:
    config = json.load(f)

connection_string = config['connectionString']


# Path to your local CSV file
container_name = 'location'
file_path = "Location_dim.csv"
blob_name = "location"

# Create a blob service client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Create a blob client using the container and blob name
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the file
with open(file_path, "rb") as data:
    blob_client.upload_blob(data)